**Name:** Cathaoir Agnew 

**ID:** 16171659


# Task 1 
**********************

In [1]:
import pandas as pd
!pip install jellyfish
import jellyfish
!wget https://norvig.com/ngrams/count_1w.txt #unigram corpus
filePath1 = "/content/count_1w.txt"
unigrams_df = pd.read_csv(filePath1,sep='\t',header=None, names=['unigram','count'])
print(f'Number of unigrams: {unigrams_df.size}')
display(unigrams_df.head(100))

--2020-11-20 12:16:49--  https://norvig.com/ngrams/count_1w.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4956241 (4.7M) [text/plain]
Saving to: ‘count_1w.txt.1’

count_1w.txt.1      100%[===================>]   4.73M  2.95MB/s    in 1.6s    

2020-11-20 12:16:52 (2.95 MB/s) - ‘count_1w.txt.1’ saved [4956241/4956241]

Number of unigrams: 666666


,unigram,count
0,the,23135851162
1,of,13151942776
2,and,12997637966
3,to,12136980858
4,a,9081174698
...,...,...
95,like,520585287
96,service,519537222
97,x,508609523
98,than,502609275


# Task 2
*******************

I have added normalised probabilities of the words and an autocorrection feature, this is not required for the task. This was more of exploration from Arash's advice and can be seen on the forum post. For now I have set the confidence probabilty quite high so it wont autocorrect, but feel free to play around with it yourself.

In [2]:
def nonWordSpellingCorrection(nonWord, threshold):
  """ Threshold is the allowed edit distance using damerau levenshtein distance that we will consider for the words. """

  print(f'nonWord = {nonWord}')

  #list to store results 
  new_word = []
  new_frequency = []
  new_probability = []
  list_of_norm_prob = []

  # play around with this value for autocorrect feature
  confidence = 0.9999

  # calculating total words of the dataframe column of count
  total_words =  unigrams_df['count'].sum()

  # now we need to look at words that are within some threshold of edit distance 
  # we will loop through the dataframe, calculate the Distance between non word & the ith  word in the data frame
  # if ith word is within a threshold of edit distance, we will add it to a new data frame, and calculate the probability of the word 

  for i in unigrams_df.unigram:

    # if you would like to see values for words that actually exist, just for fun, comment out the below 3 lines

    # first check if it is actually a non word 
    if str(i) == nonWord:
      print(f'\n"{nonWord}" is actually a word in the dictionary/dataframe.')
      return print("\nTry another word.")

    # now will calculate the edit distance between input nonWord & current word
    distance = jellyfish.damerau_levenshtein_distance(nonWord, str(i))

    # now set a threshold 
    if distance <= threshold:

      current_word_values = unigrams_df[unigrams_df.unigram == i].values

     # store the word 
      current_word = str(i)

     # store the word count 
      current_word_count = current_word_values[0][1]

      # append current word, word count, and its probability
      new_word.append(current_word)
      new_frequency.append(current_word_count)

      # word probability = current word / total number of words counts
      new_probability.append( (current_word_count / total_words) )

  # create a dataframe of the above
  new_dataframe = pd.DataFrame(list(zip(new_word, new_frequency, new_probability )) , columns = ["Word", "Frequency", 'P(word)'] )

  # going to normalize probabilites of new_dataframe 
  total_probability = new_dataframe['P(word)'].sum()

  for j in new_dataframe.Word:

    # store values of each row
    vals = new_dataframe[new_dataframe.Word == j ].values
    # storing probability for each row
    prob = vals[0][2]
    # nomralizing the probability
    norm_prob = prob / total_probability
    # append this norm probability to a list
    list_of_norm_prob.append(norm_prob)

  # this sets dataframe decimal display to 10 decimal points
  pd.set_option("display.precision", 10)
  # adding list of norm probs to the dataframe as a new column
  new_dataframe['Norm Probs'] = list_of_norm_prob

  # check to see if max norm probabilty > confidence threshold 
  if max(list_of_norm_prob) > confidence:

    # assigning the max norm probability word to the confidence_word 
    Confidence_word = new_dataframe.loc[new_dataframe['Norm Probs'].idxmax(), 'Word']
    return print(f'"{Confidence_word}" above confidence threshold probability, so "{nonWord}" autocorrected to "{Confidence_word}"')

  return new_dataframe

In [3]:
nonWordSpellingCorrection("acress", 1)

nonWord = acress


,Word,Frequency,P(word),Norm Probs
0,access,217986984,0.0003706479,0.6854260231
1,across,76597151,0.0001302397,0.2408477773
2,acres,14208905,0.0000241597,0.0446776824
3,actress,7010056,0.0000119193,0.0220420262
4,adress,984657,0.0000016742,0.0030961001
5,caress,590047,0.0000010033,0.0018553106
6,cress,279364,0.0000004750,0.0008784165
7,apress,202431,0.0000003442,0.0006365127
8,acess,171785,0.0000002921,0.0005401511


# Task 3 
******************************************


In [4]:
!pip install -U textblob

import textblob.download_corpora

Requirement already up-to-date: textblob in /usr/local/lib/python3.6/dist-packages (0.15.3)


In [5]:
from textblob import Word

def spell_check(word):
  if type(word) == str:

    w = Word(word)
    words_check = w.spellcheck()
    print("Candidates:", words_check)
    
  else: 
    print("Please insert a string")

spell_check('acress')


Candidates: [('across', 0.6851851851851852), ('access', 0.1728395061728395), ('acres', 0.1111111111111111), ('actress', 0.021604938271604937), ('caress', 0.009259259259259259)]


# Task 4 
**************************************

In [6]:
from textblob import TextBlob

def correct_spell(input_string):
  if type(input_string) == str: 
    
    #print out original input
    print(f'sentence = {input_string}')

    w = TextBlob(input_string)

    #correcting it using text blob built in function 
    corrected = w.correct()
    
    # printing out corrected sentence
    print("Corrected sentence = ", corrected)

  else: 
    print("Please insert a string")

correct_spell("I havv goood speling")

sentence = I havv goood speling
Corrected sentence =  I have good spelling
